# Iteration

In [117]:
from collections.abc import Iterable, Iterator
from datetime import date
import functools as fun

## 1. Builtin data iterable
- list
- tuple
- dict
- set
- str

In [6]:
cities = [ "Toulouse", "Lyon", "Pau", "Paris" ]
city_t = ("Toulouse", 470000, "31000")
city_dict = {
    "name": "Toulouse",
    "population": 470000,
    "cp": "31000"
}

In [4]:
for city in cities:
    print(city)

Toulouse
Lyon
Pau
Paris


In [5]:
for info in city_t:
    print(info)

Toulouse
470000
31000


In [8]:
# default iteration on dictionnaries with the keys
for info_name in city_dict:
    print(info_name)

name
population
cp


In [9]:
for info_name in city_dict.keys():
    print(info_name)

name
population
cp


In [10]:
for info_value in city_dict.values():
    print(info_value)

Toulouse
470000
31000


In [11]:
for info_name, info_value in city_dict.items():
    print(info_name, "->", info_value)

name -> Toulouse
population -> 470000
cp -> 31000


In [12]:
city = cities[0]
for letter in city:
    print(letter)

T
o
u
l
o
u
s
e


In [13]:
type(cities)

list

In [15]:
# # TypeError: 'int' object is not iterable
# for x in 123:
#     pass

In [16]:
cities.__iter__

<method-wrapper '__iter__' of list object at 0x00000251D6CEEF80>

In [25]:
# duck typing: presence of method __iter__ 
for data in cities, city_t, city_dict, city:
    isIterable1 = '__iter__' in dir(data)
    isIterable2 = isinstance(data, Iterable)
    print(
        f"data: {data}", 
        f"type: {type(data)}", 
        f"iterable (duck typing): {isIterable1}",
        f"iterable (inheritance): {isIterable2}",
        sep="\n\t- "
    ) 

data: ['Toulouse', 'Lyon', 'Pau', 'Paris']
	- type: <class 'list'>
	- iterable (duck typing): True
	- iterable (inheritance): True
data: ('Toulouse', 470000, '31000')
	- type: <class 'tuple'>
	- iterable (duck typing): True
	- iterable (inheritance): True
data: {'name': 'Toulouse', 'population': 470000, 'cp': '31000'}
	- type: <class 'dict'>
	- iterable (duck typing): True
	- iterable (inheritance): True
data: Toulouse
	- type: <class 'str'>
	- iterable (duck typing): True
	- iterable (inheritance): True


### NB: formated string

In [29]:
day = date.today()
day

datetime.date(2023, 6, 19)

In [31]:
"city: {}, population: {}, day: {:%d/%m/%Y}".format(city, city_t[1], day)

'city: Toulouse, population: 470000, day: 19/06/2023'

In [32]:
f"city: {city}, population: {city_t[1]}, day: {day:%d/%m/%Y}"

'city: Toulouse, population: 470000, day: 19/06/2023'

## 2. Generator, lazy evaluation

In [43]:
def generator_cities():
    return (c.upper() for c in cities)

In [44]:
g = generator_cities()
g

<generator object generator_cities.<locals>.<genexpr> at 0x00000251D720A5A0>

In [45]:
isinstance(g, Iterable)

True

In [46]:
for c in g:
    print(c)

TOULOUSE
LYON
PAU
PARIS


In [47]:
g = generator_cities()
l = list(g)
l

['TOULOUSE', 'LYON', 'PAU', 'PARIS']

In [48]:
g = generator_cities()
max(g)

'TOULOUSE'

In [49]:
# list comprhension
[ c.upper() for c in cities ]

['TOULOUSE', 'LYON', 'PAU', 'PARIS']

### Fibonacci infinite

In [56]:
def fibo_infinite():
    a = 0
    b = 1
    yield a
    yield b
    while True:
        a, b = b, a + b
        yield b

In [57]:
g = fibo_infinite()
g

<generator object fibo_infinite at 0x00000251D720B100>

In [58]:
for _ in range(10):
    v = next(g)
    print(v)

0
1
1
2
3
5
8
13
21
34


In [64]:
# execute this cell as many times as you want
next(g)

233

### Fibonacci finite

In [62]:
def fibo_n(n):
    """ generate n values of Fibonacci series"""
    a = 0
    b = 1
    if n > 0:
        yield a
    if n > 1:
        yield b
    for _ in range(n-2):
        a, b = b, a + b
        yield b

In [72]:
print(list(fibo_n(0)))
print(list(fibo_n(1)))
print(list(fibo_n(2)))
print(list(fibo_n(10)))

[]
[0]
[0, 1]
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


## 3. Iterator/Iterable

In [93]:
it = iter(cities)
assert '__next__' in dir(it) # duck typing
assert isinstance(it, Iterator) # inheritance
assert isinstance(it, Iterable) # inheritance
it

In [89]:
while True:
    try:
        city = next(it)
        print("Next city:", city)
    except StopIteration:
        break

Next city: Toulouse
Next city: Lyon
Next city: Pau
Next city: Paris


In [90]:
it2 = iter(it)
assert  it2 is it
it2

In [92]:
# an iterator is iterable, so any function with an arg iterable can accept it
it = iter(cities)
max(it)

'Toulouse'

In [98]:
cities_with_extra = list(
    zip(
        range(1_000_000_000_000_000),     # lazy + iterable
        cities,                           # iterable
        (len(city) for city in cities),   # generator: iterator/iterable
        iter(cities[0])                   # iterator/iterable
    )
)
cities_with_extra

[(0, 'Toulouse', 8, 'T'),
 (1, 'Lyon', 4, 'o'),
 (2, 'Pau', 3, 'u'),
 (3, 'Paris', 5, 'l')]

In [97]:
# range is another lazy object, iterable
range(10)

range(0, 10)

#### Builtin functions using Iterable or Iterator
 - iter, next: mechanism
 - all, any: iterable -> bool
 - sum, min, max: iterable -> val
 - map, filter (can be expressed with a for expression)
 - range
 - list, tuple, dict: iterable -> container
 - sorted: iterable -> list

### Atelier builtin iterable
1. Est-ce que toutes les villes ont au moins 3 lettres ?
2. Y a t il une ville avec 8 lettres ?
3. Combien de lettres pour écrire toutes les villes (Scrabble) ?
4. liste des villes commençant par P
5. Trier les villes par ordre alphabétique et numéroter les résultats de 1 à n

In [100]:
# 1
r1 = all(len(c) >= 3 for c in cities)
print("r1:",r1)

r1: True


In [105]:
# 2
any((len(city) == 8) for city in cities)

True

In [108]:
# 3bis
# ensemble des lettres constitituant les villes (sans répétition)
full_str = ""
for city in cities:
    full_str += city

set_full_str_case = set(full_str)
question3 = set(full_str.lower())
print(set_full_str_case)
print(question3)

{'n', 'r', 'l', 'e', 'o', 'L', 'a', 'i', 'u', 'P', 'T', 'y', 's'}
{'n', 'r', 'l', 'e', 'p', 'o', 'i', 'a', 'u', 'y', 't', 's'}


In [123]:
# non optimized reduction (generate a new set at each iteration
letters = fun.reduce(set.union, (set(city.lower()) for city in cities), set())
letters

{'a', 'e', 'i', 'l', 'n', 'o', 'p', 'r', 's', 't', 'u', 'y'}

In [124]:
# 3
sum(len(city) for city in cities)

20

In [127]:
# NB: an expression for must be parenthesized if more than one arg
sum((len(city) for city in cities), 100)

120

In [128]:
[ len(city) for city in cities ]

[8, 4, 3, 5]

In [130]:
# bad practice ! memory peak
sum([ len(city) for city in cities ])

20

In [133]:
# 4 - sol imperative
res4 = []
for city in cities:
    if city[0]=="P":
        res4.append(city)
        print("Cette ville commence par P")
    else: 
        print("Cette ville ne commence pas par P")
res4

Cette ville ne commence pas par P
Cette ville ne commence pas par P
Cette ville commence par P
Cette ville commence par P


['Pau', 'Paris']

In [135]:
# 4 - sol map/reduce with list comprehension
[city for city in cities if city[0] == 'P']

['Pau', 'Paris']

In [138]:
list(filter(lambda c: c.startswith('P'), cities))

['Pau', 'Paris']

In [139]:
list(map(str.upper, filter(lambda c: c.startswith('P'), cities)))

['PAU', 'PARIS']

In [140]:
[ c.upper() for c in cities if  c.startswith('P') ]

['PAU', 'PARIS']

In [141]:
city = "Toulouse"
print(city.upper())
print(str.upper(city))

TOULOUSE
TOULOUSE


In [143]:
# 5
sorted_cities = sorted(cities)
question5 = [(i+1,city) for i, city in enumerate(sorted_cities)]
question5


[(1, 'Lyon'), (2, 'Paris'), (3, 'Pau'), (4, 'Toulouse')]

In [146]:
# enumerate can start at any value (2nd arg)
sorted_cities = sorted(cities)
question5 = list(enumerate(sorted_cities, 1))
question5

[(1, 'Lyon'), (2, 'Paris'), (3, 'Pau'), (4, 'Toulouse')]

In [ ]:
# enumerate is equivalent to zip + range
list(zip(range(1, len(cities)+1), sorted(cities)))